In [ ]:
import pandas as pd
import os
import numpy as np

# Загрузка данных
payments_files = ['data/payments_1.csv', 'data/payments_2.csv']
providers_files = ['data/providers_1.csv', 'data/providers_2.csv']

com_w = 1
conv_w = 0
avg_time_w = 0

def calc_rating(row):
    rating = com_w * row['COMMISSION_NORM'] + conv_w * row['CONVERSION_NORM'] + avg_time_w * row['AVG_TIME_NORM']
    return rating


def chose_providers(row):
    tran_time = row['eventTimeRes'].floor('H')
    tran_cur = row['cur']
    tran_amount_in_usd = row['amount_in_usd']
    rating = ratings[(ratings['TIME']==tran_time) & (ratings['cur']==tran_cur)]

    # Получаем список ID для сортировки
    sorting_order = rating['ID'].values[0]

    # Добавляем условие фильтрации
    available_providers = providers_final_df[
        (providers_final_df['TIME'] == rating['TIME'].values[0]) &
        (providers_final_df['CURRENCY'] == rating['cur'].values[0]) &
        (providers_final_df['ID'].isin(sorting_order)) &
        (providers_final_df['MIN_SUM_USD'] <= tran_amount_in_usd) &
        (providers_final_df['MAX_SUM_USD'] >= tran_amount_in_usd)
    ]

    available_providers = available_providers.merge(provider_day_data,how='left',
                              left_on=['day','ID','CURRENCY'],
                              right_on=['day','ID','CURRENCY']
    )

    #Проверяем что есть хотябы один провайдер, который может пропустить транзакцию не превысив максимальный лимит
    available_providers = available_providers[available_providers['DAY_TRAN_AMOUNT']+tran_amount_in_usd < available_providers['LIMIT_MAX_USD']]

    if available_providers.empty:
        return pd.Series([None, None, None, None], index=['flow', 'avg_time', 'conv', 'comm'])

    # Сортируем DataFrame по 'ID' в порядке из sorting_order
    available_providers['ID'] = pd.Categorical(available_providers['ID'], categories=sorting_order, ordered=True)
    available_providers = available_providers.sort_values(by='ID')
    available_providers.reset_index(inplace=True)

    available_providers['calculated'] = available_providers['CONVERSION']  # Копируем значение в новую колонку

    # Используем цикл для итеративного вычисления
    for i in range(1, len(available_providers)):
        previous_sum = available_providers.loc[:i-1, 'calculated'].sum()  # Сумма всех предыдущих значений
        available_providers.loc[i, 'calculated'] = (1 - previous_sum) * available_providers.loc[i, 'CONVERSION']

    def update_day_tran_amount(row):
        filtr = (
            (provider_day_data['ID'] == row['ID']) &
            (provider_day_data['day'] == row['day']) &
            (provider_day_data['CURRENCY'] == row['CURRENCY'])
        )
        provider_day_data.loc[filtr, 'DAY_TRAN_AMOUNT'] += row['calculated'] * tran_amount_in_usd

    available_providers.apply(update_day_tran_amount, axis=1)


    flow = '-'.join(map(str, sorting_order))
    avg_time = available_providers.iloc[0]['AVG_TIME']
    conv = available_providers.iloc[0]['CONVERSION']
    comm = available_providers.iloc[0]['COMMISSION']
    return pd.Series([flow, avg_time, conv, comm], index=['flow', 'avg_time', 'conv', 'comm'])

def chose_providers_to_calc_w(row):
    tran_time = row['eventTimeRes'].floor('H')
    tran_cur = row['cur']
    tran_amount_in_usd = row['amount_in_usd']
    rating = ratings[(ratings['TIME']==tran_time) & (ratings['cur']==tran_cur)]
    # print(rating['ID'].values,type(rating['ID'].values))

    # Получаем список ID для сортировки
    sorting_order = rating['ID'].values[0]

    # Добавляем условие фильтрации
    available_providers = providers_final_df[
        (providers_final_df['TIME'] == rating['TIME'].values[0]) &
        (providers_final_df['CURRENCY'] == rating['cur'].values[0]) &
        (providers_final_df['ID'].isin(sorting_order)) &
        (providers_final_df['MIN_SUM_USD'] <= tran_amount_in_usd) &
        (providers_final_df['MAX_SUM_USD'] >= tran_amount_in_usd)
    ]

    if available_providers.empty:
        return pd.Series([None, None, None, None], index=['flow', 'avg_time', 'conv', 'comm'])

    # Сортируем DataFrame по 'ID' в порядке из sorting_order
    available_providers['ID'] = pd.Categorical(available_providers['ID'], categories=sorting_order, ordered=True)
    available_providers = available_providers.sort_values(by='ID')
    available_providers.reset_index(inplace=True)

    flow = '-'.join(map(str, sorting_order))
    avg_time = available_providers.iloc[0]['AVG_TIME']
    conv = available_providers.iloc[0]['CONVERSION']
    comm = available_providers.iloc[0]['COMMISSION']
    return pd.Series([flow, avg_time, conv, comm], index=['flow', 'avg_time', 'conv', 'comm'])


def generate_weights(n, seed=None):
    """
    Генерирует n наборов весов com_w, conv_w, avg_time_w, таких что com_w + conv_w + avg_time_w = 1.
    Добавляет к результату три крайние точки: [1, 0, 0], [0, 1, 0], [0, 0, 1].

    :param n: Количество случайных наборов весов.
    :param seed: Значение для генератора случайных чисел (для воспроизводимости).
    :return: DataFrame с колонками com_w, conv_w, avg_time_w.
    """
    if seed is not None:
        np.random.seed(seed)  # Устанавливаем seed для воспроизводимости

    weights = []
    for _ in range(n):
        w = np.random.dirichlet([1, 1, 1])  # Используем распределение Дирихле
        weights.append(w)

    # Добавляем крайние точки
    extreme_points = [[1, 0, 0], [0, 1, 0], [0, 0, 1]]
    weights.extend(extreme_points)

    return pd.DataFrame(weights, columns=['com_w', 'conv_w', 'avg_time_w'])


for payments_file, providers_file in zip(payments_files, providers_files):
    # Чтение текущих платежей и провайдеров
    payments_df = pd.read_csv(payments_file)
    payments_df['eventTimeRes'] = pd.to_datetime(payments_df['eventTimeRes'])

    providers_df = pd.read_csv(providers_file)

    max_index = providers_df.groupby(['ID', 'TIME', 'CURRENCY']).apply(lambda x: x.index.max())
    providers_df = providers_df.loc[max_index].reset_index(drop=True)
    #####

    providers_df['TIME'] = pd.to_datetime(providers_df['TIME'])
    # Чтение файла с обменными курсами
    ex_rates = pd.read_csv('data/ex_rates.csv')

    # Переименование колонки 'destination' в 'cur'
    ex_rates.rename(columns={'destination': 'cur'}, inplace=True)

    # Добавление полей MIN_SUM_USD, MAX_SUM_USD, LIMIT_MIN_USD, LIMIT_MAX_USD в providers_df
    providers_df = providers_df.merge(ex_rates[['cur', 'rate']], left_on='CURRENCY', right_on='cur', how='left')


###########################################################################

    # Находим дневние лимиты
    providers_df['day'] = providers_df['TIME'].dt.date

    # Группируем по 'ID', 'TIME', 'CURRENCY' и находим индекс с максимальным значением
    max_index = providers_df.groupby(['ID', 'TIME', 'CURRENCY']).apply(lambda x: x.index.max())
    providers_df = providers_df.loc[max_index].reset_index(drop=True)

    # Находим дневние лимиты
    firt_provider_info_in_the_day = (
        providers_df.groupby(['day','cur', 'ID'])['TIME']
        .min()
        .reset_index(name="earliest_time")
    )

    # Объединяем данные через merge
    filter_df = firt_provider_info_in_the_day[['ID','cur', 'earliest_time']]
    result = providers_df.merge(
        filter_df,  # Соединяем с фильтром
        left_on=['ID','cur', 'TIME'],  # Сравниваем по ID и TIME
        right_on=['ID','cur', 'earliest_time'],  # Сравниваем с ID и earliest_time
        how='inner'  # Оставляем только совпадения
    )

    day_limits=result[['day','ID','cur','LIMIT_MIN','LIMIT_MAX']].rename(columns={'LIMIT_MIN': 'DAY_LIMIT_MIN',
                                                'LIMIT_MAX': 'DAY_LIMIT_MAX'})
    providers_df = providers_df.merge(
        day_limits,
        left_on=['ID','cur', 'day'],
        right_on=['ID','cur', 'day'],
        how='left'
    )

    # Расчет MIN/MAX в USD
    providers_df['MIN_SUM_USD'] = providers_df['MIN_SUM'] * providers_df['rate']
    providers_df['MAX_SUM_USD'] = providers_df['MAX_SUM'] * providers_df['rate']
    providers_df['LIMIT_MIN_USD'] = providers_df['DAY_LIMIT_MIN'] * providers_df['rate']
    providers_df['LIMIT_MAX_USD'] = providers_df['DAY_LIMIT_MAX'] * providers_df['rate']
    providers_df = providers_df.drop(['DAY_LIMIT_MIN', 'DAY_LIMIT_MAX'], axis=1)


#############################################################

    # Объединение payments_df с ex_rates по валюте
    payments_df = payments_df.merge(ex_rates, left_on='cur', right_on='cur', how='left')

    # Проверка наличия обменных курсов для всех валют
    missing_rates = payments_df[payments_df['rate'].isnull()]['cur'].unique()
    if len(missing_rates) > 0:
        print(f"Внимание: Отсутствуют обменные курсы для валют: {missing_rates}")

    # Валидация типов данных
    payments_df['amount'] = pd.to_numeric(payments_df['amount'], errors='coerce')
    payments_df['rate_to_usd'] = pd.to_numeric(payments_df['rate'], errors='coerce')

    # Вычисление суммы в USD
    payments_df['amount_in_usd'] = payments_df['amount'] * payments_df['rate']

    # Создание DataFrame с уникальными временами
    unique_times = providers_df[['TIME']].drop_duplicates().reset_index(drop=True)

    # Сджойнить unique_times с providers_df по условию времени (SQL аналог: select * from unique_time ut left join providers_df p on ut.TIME >= p.TIME)
    unique_times['key'] = 1
    providers_df['key'] = 1
    providers_final_df = unique_times.merge(providers_df, on='key')
    providers_final_df = providers_final_df[providers_final_df['TIME_x'] >= providers_final_df['TIME_y']].drop(columns=['key'])

    # Переименовываем колонки для читаемости
    providers_final_df.rename(columns={'TIME_x': 'TIME_unique', 'TIME_y': 'TIME_providers'}, inplace=True)



    providers_final_df = providers_final_df.sort_values(by='TIME_providers').groupby(['TIME_unique','ID'], as_index=False).last()
    providers_final_df.drop(columns=['TIME_providers'], inplace=True)
    providers_final_df.rename(columns={'TIME_unique': 'TIME'}, inplace=True)

    #Делаем дф для подсчета сколько "провели" через провайдера денег
    provider_day_data= providers_final_df[['day','ID','CURRENCY','LIMIT_MIN_USD']].copy()
    provider_day_data = provider_day_data.drop_duplicates().reset_index(drop=True)
    provider_day_data['FINE'] = provider_day_data['LIMIT_MIN_USD']*0.01
    provider_day_data['DAY_TRAN_AMOUNT'] = 0.0
    provider_day_data = provider_day_data.drop(['LIMIT_MIN_USD'], axis=1)

    #---------------------------------
    # Нормализация столбцов
    providers_final_df['COMMISSION_NORM'] = (providers_final_df['COMMISSION'] - providers_final_df['COMMISSION'].min()) / (providers_final_df['COMMISSION'].max() - providers_final_df['COMMISSION'].min())
    providers_final_df['CONVERSION_NORM'] = (1 - providers_final_df['CONVERSION'] - (1 - providers_final_df['CONVERSION']).min()) / ((1 - providers_final_df['CONVERSION']).max() - (1 - providers_final_df['CONVERSION']).min())
    providers_final_df['AVG_TIME_NORM'] = (providers_final_df['AVG_TIME'] - providers_final_df['AVG_TIME'].min()) / (providers_final_df['AVG_TIME'].max() - providers_final_df['AVG_TIME'].min())

    ################
    #Выбираем оптимальные веса
    ################
    sample_payments = payments_df.sample(frac=0.05,random_state=42)
    weights_df = generate_weights(10, seed=42)

    for i, weights in enumerate(weights_df.itertuples(index=False)):
        print('Сейчас итерация ',i)
        com_w, conv_w, avg_time_w = weights.com_w, weights.conv_w, weights.avg_time_w
        # Ваш код с расчетом sample_payments
        providers_final_df['rating'] = providers_final_df.apply(calc_rating,axis=1)
        ratings = (
            providers_final_df.sort_values(by=['TIME', 'rating'], ascending=[True, True])
            .groupby(['TIME','cur'])['ID']
            .apply(list)
            .reset_index()
        )
        sample_payments[['flow', 'avg_time', 'conv', 'comm']] = sample_payments.apply(chose_providers_to_calc_w, axis=1)
        sample_payments['1_minus_conv'] = 1 - sample_payments['conv']  # Вероятность неуспеха

        # Нормализация (приведение к диапазону [0, 1])
        sample_payments['avg_time_norm'] = (sample_payments['avg_time'] - sample_payments['avg_time'].min()) / (sample_payments['avg_time'].max() - sample_payments['avg_time'].min())
        sample_payments['1_minus_conv_norm'] = (sample_payments['1_minus_conv'] - sample_payments['1_minus_conv'].min()) / (sample_payments['1_minus_conv'].max() - sample_payments['1_minus_conv'].min())
        sample_payments['comm_norm'] = (sample_payments['comm'] - sample_payments['comm'].min()) / (sample_payments['comm'].max() - sample_payments['comm'].min())

        # Считаем средние показатели
        avg_time_mean = sample_payments['avg_time_norm'].mean()
        conv_mean = sample_payments['1_minus_conv_norm'].mean()
        comm_mean = sample_payments['comm_norm'].mean()

        # Вычисление длины вектора
        vector_length = np.sqrt(avg_time_mean**2 + conv_mean**2 + comm_mean**2)

        # Записываем длину вектора в weights_df
        weights_df.loc[i, 'vector_length'] = vector_length

    min_vector_index = weights_df['vector_length'].idxmin()
    com_w,conv_w,avg_time_w = weights_df.loc[min_vector_index, ['com_w', 'conv_w', 'avg_time_w']]
    print("Оптимальные веса комиссия, конверсия, среднее время:")
    print(com_w,conv_w,avg_time_w)
    #---------------------------------

    providers_final_df['rating'] = providers_final_df.apply(calc_rating,axis=1)

    #Считаем рэйтинги провайдеров
    ratings = (
        providers_final_df.sort_values(by=['TIME', 'rating'], ascending=[True, True])
        .groupby(['TIME','cur'])['ID']
        .apply(list)
        .reset_index()
    )



    payments_df[['flow','avg_time','conv','comm']]= payments_df.apply(chose_providers,axis=1)

    os.makedirs('data/final', exist_ok=True)
    payments_df[['eventTimeRes','amount','cur','payment','cardToken','flow']].to_csv('data/final/'+os.path.basename(payments_file))